# Graph Analysis
### A* algorithm
**A\* algorithm** is an **informed** search algorithm that finds the shortest path from a start node to a goal node by combining:
- Actual cost from start ($g(n)$) - like Dijkstra
- Heuristic estimate to goal ($h(n)$) - greedy guidance

We may express A* algorithm for a given **graph**, **start** node, and **goal** node as:

1. **Initialize**:  
   - `g(start) = 0`, `f(start) = h(start)`  
   - Add `start` to open set (min-heap by `f`)

2. **While open set not empty**:  
   a. Pop node `current` with lowest `f`  
   b. If `current == goal`, reconstruct and return path  
   c. For each neighbor:  
      - `tentative_g = g(current) + cost`  
      - If better path found:  
        • Update `g`, `f`, and `came_from`  
        • Push neighbor to open set

3. **If goal not reached**, return **no path**

> Here, we know that:  
> - `g(n)` = cost from start to `n`  
> - `h(n)` = heuristic to goal  
> - `f(n) = g(n) + h(n)`

<hr>

Some key points about A* algorithm for a graph $G=(V,E)$:
- **Time Complexity:** O((|V|+|E|) log |V|) but with a good heuristic it is much less
- **Space complexity:** O(|V|+|E|)
- **Optimality:** Guarantees shortest paths with non-negative weights with an admissible heuristic
- **Completeness:** Always finds a solution if one exists in finite graphs

**Reminder:** **Completeness** in search algorithms refers to whether the algorithm is guaranteed to find a solution if one exists.

<hr>

In the following, we implement A* algorithm with a min-heap as a priority queue (similar to the Dijkstra's algorithm). Then, we use if for a simple graph and find the shortest path from a start node to a goal node. 

https://github.com/ostad-ai/Graph-Analysis
<br>Explanation in English :https://www.pinterest.com/HamedShahHosseini/graph-analysis/

In [1]:
# import required modules
import heapq
import math

In [2]:
# Common Heuristic Functions
def manhattan_distance(a, b):
    """Manhattan distance for grid-based problems"""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def euclidean_distance(a, b):
    """Euclidean distance for straight-line problems"""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def chebyshev_distance(a, b):
    """Also known as maximum metric, it is for 8-direction grids"""
    return max(abs(a[0]-b[0]),abs(a[1]-b[1]))

In [3]:
def reconstruct_path(came_from, current):
    """Reconstruct path from start to current node"""
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    return path[::-1]  # Reverse to get start->goal

In [4]:
def a_star(graph, start, goal, heuristic):
    """
    A* Search Algorithm implementation
    
    Args:
        graph: dict {node: [(neighbor, cost), ...]}
        start: starting node
        goal: target node
        heuristic: function(node, goal) -> estimated cost
    
    Returns:
        path: list of nodes from start to goal
        total_cost: actual cost of path
    """
    
    # Priority queue: (f_score, node)
    open_set = []
    heapq.heappush(open_set, (heuristic(start, goal), start))
    
    # For path reconstruction
    came_from = {}
    
    # Cost from start to each node
    g_score = {start: 0}
    
    # Estimated total cost (g + h)
    f_score = {start: heuristic(start, goal)}
    
    while open_set:
        # Get node with lowest f_score
        current_f, current = heapq.heappop(open_set)
        
        # Early exit if goal reached
        if current == goal:
            return reconstruct_path(came_from, current), g_score[current]
        
        # Explore neighbors
        for neighbor, cost in graph.get(current, []):
            # Calculate tentative g_score
            tentative_g = g_score[current] + cost
            
            # If this path to neighbor is better
            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                # This path is better, record it
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score[neighbor] = tentative_g + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score[neighbor], neighbor))
    
    return None, float('inf')  # No path found

In [5]:
"""A* on a grid with actual obstacles that block movement"""
# 0 = free cell, 1 = obstacle
grid = [
    [0, 0, 0, 1, 0],  # Row 0
    [0, 1, 0, 1, 0],  # Row 1  
    [0, 1, 0, 0, 0],  # Row 2
    [0, 0, 0, 1, 0],  # Row 3
    [0, 1, 1, 1, 0]   # Row 4
]

# Build graph from grid - only include traversable cells
grid_graph = {}

for x in range(len(grid)):
    for y in range(len(grid[0])):
        if grid[x][y] == 1:  # Skip obstacles
            continue

        neighbors = []
        # Check all 4 directions
        for dx, dy in [(0,1), (1,0), (0,-1), (-1,0)]:
            newx, newy = x + dx, y + dy
            if (0 <= newx < len(grid) and 0 <= newy < len(grid[0]) 
                and grid[newx][newy] == 0):  # Not an obstacle
                neighbors.append(((newx, newy), 1))  # Cost 1 to move

        grid_graph[(x, y)] = neighbors

start = (0, 0)
goal = (4, 4)

path, cost = a_star(grid_graph, start, goal, manhattan_distance)

print("Grid WITH Obstacles Example:")
print(f"Start: {start}, Goal: {goal}")
print(f"Path: {path}")
print(f"Total cost: {cost}")
print('-'*50)
print("Grid visualization (O=obstacle, .=free, S=start, G=goal):")

# Visualize the grid and path
for x in range(len(grid)):
    row = []
    for y in range(len(grid[0])):
        if (x, y) == start:
            row.append('S')
        elif (x, y) == goal:
            row.append('G')
        elif grid[x][y] == 1:
            row.append('O')
        elif path and (x, y) in path:
            row.append('*')  # Path marker
        else:
            row.append('.')
    print(' '.join(row))

Grid WITH Obstacles Example:
Start: (0, 0), Goal: (4, 4)
Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]
Total cost: 8
--------------------------------------------------
Grid visualization (O=obstacle, .=free, S=start, G=goal):
S * * O .
. O * O .
. O * * *
. . . O *
. O O O G
